# Simple Example with map_partitions

In this tutorial, we will demonstrate how to perfom a user-defined-function on all partitions of an LSDB catalog.

## Introduction

LSDB/HATS catalogs are organized into partitions, and this number of partitions is reported as `npartitions=` in the header, whenever printing a catalog.

The `map_partitions` method provides a means for users to execute their own analysis functions on each *partition* of the catalog data. The data will be passed to your function as a dataframe.

Partitions will be processed in parallel.
This can be used to distribute operations across the catalog with maximum efficiency.

## 1. Load a catalog

We will use a small cone of the Gaia DR3 catalog, and only load the columns that we are interested in. This limits the overall memory requirements of the pipeline.

In [1]:
# Dask puts out more advisory logging that we care for.
# It takes some doing to quiet all of it, but this recipe works.

import dask

dask.config.set({"logging.distributed": "critical"})

import logging

# This also has to be done, for the above to be effective
logger = logging.getLogger("distributed")
logger.setLevel(logging.CRITICAL)

import warnings

# Finally, suppress the specific warning about Dask dashboard port usage
warnings.filterwarnings("ignore", message="Port 8787 is already in use.")

In [2]:
import lsdb

gaia_root = "https://data.lsdb.io/hats"
gaia3 = lsdb.open_catalog(
    f"{gaia_root}/gaia_dr3/gaia",
    margin_cache=f"{gaia_root}/gaia_dr3/gaia_10arcs",
    search_filter=lsdb.ConeSearch(ra=280, dec=-60, radius_arcsec=2 * 3600),
    columns=[
        "source_id",
        "ra",
        "dec",
        "phot_g_mean_mag",
    ],
)
gaia3

,source_id,ra,dec,phot_g_mean_mag
npartitions=4,,,,
"Order: 4, Pixel: 2944",int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 4, Pixel: 2945",...,...,...,...
"Order: 4, Pixel: 2946",...,...,...,...
"Order: 4, Pixel: 2947",...,...,...,...


In [3]:
# You can get the number of partitions programmatically this way.
# This can be valuable when you want to choose the optimal number
# of workers to process the partitions.
gaia3.npartitions

4

In [4]:
# You can also access the individual HealpixPixel objects for
# each partition, this way, inspecting their order and pixel,
# if desired.
px = gaia3.get_healpix_pixels()[0]
px.order, px.pixel

(np.int64(4), np.int64(2944))

## 2. Generating New Columns

Since the partition's `DataFrame` is passed in to your custom function, you can augment it with new columns based on the existing columns, in ordinary Pandas style.

### 2.1 What you can map

The trick is understanding what kind of custom function you can pass to `.map_partitions`.  Your function is going to receive a Pandas DataFrame as its first parameter.  Other parameters can be passed in as keyword arguments to `.map_partitions`, as you'll see later on.  For now, we'll use a function that takes in one partition and produces a result that has the same shape.

Because the catalog is loaded lazily, `.map_partitions` also returns a lazy, or unevaluated, result.  You can see the results the same way you can realize the original catalog, by any of these means:
  * calling `.compute()` to produce a `DataFrame` in memory;
  * calling `.to_hats()` to serialize it to disk as a HATS-format file;
  * calling `.head()` to see the first few rows, `.tail()` to see the last few.

In [5]:
def mean_sq(df, pixel):
    df["phot_g_mean_mag_sq"] = df["phot_g_mean_mag"] ** 2
    return df


unrealized = gaia3.map_partitions(mean_sq, include_pixel=True)
unrealized

,source_id,ra,dec,phot_g_mean_mag,phot_g_mean_mag_sq
npartitions=4,,,,,
"Order: 4, Pixel: 2944",int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 4, Pixel: 2945",...,...,...,...,...
"Order: 4, Pixel: 2946",...,...,...,...,...
"Order: 4, Pixel: 2947",...,...,...,...,...


Taking a quick peek to see whether our function works correctly, and if the results in our new column are about what we expect:

In [6]:
unrealized.head()

,source_id,ra,dec,phot_g_mean_mag,phot_g_mean_mag_sq
_healpix_29,,,,,
3315212135629220059,6630424242158614528,279.475941,-61.973682,20.157476,406.323839
3315212197603296958,6630424379597543936,279.40738,-61.977054,19.488537,379.803074
3315212213755151065,6630424413957281792,279.436314,-61.979345,18.925087,358.158918
3315212218438134563,6630424418254234752,279.412417,-61.979072,20.522877,421.18848
3315212218704706046,6630424418256341376,279.416172,-61.977445,16.336718,266.888355


Looks good! Now on to computing the complete result.

This unrealized result has a top-level property indicating how many partitions it has.  We can use this to choose our number of workers directly.

However, it's a good idea to bound the number of workers, in case the number of partitions is larger than we expect (or we move this code fragment elsewhere).

In [7]:
%%time
npartitions = gaia3.npartitions
from dask.distributed import Client

with Client(n_workers=min(8, npartitions)) as client:
    result = unrealized.compute()
result

CPU times: user 1.24 s, sys: 846 ms, total: 2.09 s
Wall time: 23.6 s


,source_id,ra,dec,phot_g_mean_mag,phot_g_mean_mag_sq
_healpix_29,,,,,
3315212135629220059,6630424242158614528,279.475941,-61.973682,20.157476,406.323839
3315212197603296958,6630424379597543936,279.40738,-61.977054,19.488537,379.803074
...,...,...,...,...,...
3318663093295462547,6637326190180387584,280.426265,-58.015067,21.097507,445.104802
3318663093808668415,6637326185883831296,280.423503,-58.013744,20.301455,412.149075


No reduction step is needed here since the operation is not a reducing operation.
There are as many rows in the new output as there were in the input.

## 3. Functions that reduce

The above works when your output rows are the same as your input rows.  When you're doing a reducing operation (such as calculating statistics), the process changes a little.

### 3.1. Your function's parameters

Again, your first input parameter is a `pd.DataFrame` that is one partition of the catalog, and the return value of your function needs to be the same, even if your result has only a single row.

If you want to know the HEALPix number of the partition, calling `.map_partitions` with `include_pixel=True` will pass that as the second parameter to your function.  We'll do this in this example, for demonstration purposes, though it isn't strictly necessary to this task.

If you have any other parameters that your function requires, take them as keyword arguments, and you can pass their values in as such, when calling `.map_partitions`.  Our example will do this, too, taking a `target_column=` argument.

### 3.2. What you get back

The operation we're going to do here is a reducing operation (min and max), and it will be run on each partition, reducing the many rows in each partition to a single value.  This means that the output of `.map_partitions` in this case will contain *one row per partition*.  Thus, you will need to do additional reduction on this output in order to get a single final result.

In [8]:
# Note that this function must work correctly when given an empty DataFrame
# as an input, too; if not, you're obliged to provide "meta", that is,
# information about output type and shape.
import pandas as pd


def find_stats(df, pixel, target_column=""):
    c = df[target_column]
    min_val = c.min()
    max_val = c.max()
    mean_val = c.mean()
    return pd.DataFrame(
        [
            {
                "pixel": pixel,
                f"{target_column}_min": min_val,
                f"{target_column}_max": max_val,
                f"{target_column}_mean": mean_val,
            }
        ]
    )

### 3.3 When You Need `meta=`

The above definition of `find_stats` works even with an empty `DataFrame` argument because `.mean()` is written to handle zero-row inputs without errors.

But your own custom function might not. As a trivial example, suppose you implemented the arithmetic mean yourself, as below.

In [9]:
# This version of the function will NOT work with map_partitions as is, because
# of the attempt to divide by `c.count()`, which will be zero for an empty input.
import pandas as pd


def find_stats_needs_meta(df, pixel, target_column=""):
    c = df[target_column]
    min_val = c.min()
    max_val = c.max()
    # WARNING! c.count() == 0 when passed an empty DataFrame.
    # But meta= will come to the rescue.
    mean_val = c.sum() / c.count()
    return pd.DataFrame(
        [
            {
                "pixel": pixel,
                f"{target_column}_min": min_val,
                f"{target_column}_max": max_val,
                f"{target_column}_mean": mean_val,
            }
        ]
    )

In the above case, then, you need to indicate to Dask what type the output will be.

What Dask needs to know are the column names and their order, and so the below definition works, even though the types of the columns aren't indicated. (The type of `"pixel"` will default to `float64`, which is wrong, but doesn't matter in this case.)

In [10]:
output_meta = pd.DataFrame(
    {
        "pixel": [],
        "phot_g_mean_mag_min": [],
        "phot_g_mean_mag_max": [],
        "phot_g_mean_mag_mean": [],
    }
)

Here's another definition of `output_meta` that works equally well. The type of `"pixel"` will now be more correct:

In [11]:
output_meta = pd.DataFrame(
    [
        {
            "pixel": lsdb.HealpixPixel(0, 0),
            "phot_g_mean_mag_min": 0.0,
            "phot_g_mean_mag_max": 0.0,
            "phot_g_mean_mag_mean": 0.0,
        }
    ]
)

In [12]:
output_meta["pixel"].dtype

dtype('O')

although `DataFrame` only cares that it is an "Object", as we can see.

These can be complicated and error-prone to construct, and small mistakes create confusing errors that show up late in the computation.

To help with these difficulties, Dask does provide a `make_meta` function. If you can pass it a single valid row from your catalog (that's what `.head(1)` will do) which will work with your custom function, `make_meta` will generate the meta for it.

In [13]:
from dask.dataframe.utils import make_meta

output_meta = make_meta(
    find_stats(gaia3.head(1), gaia3.get_healpix_pixels()[0], target_column="phot_g_mean_mag")
)

Passing a correct `meta=` to `.map_partitions` will allow Dask to skip sending your function an empty `DataFrame`, and so, in our case of `find_stats_needs_meta` (where we depend on a non-zero `c.count()`), it will succeed without error.

In [14]:
unrealized = gaia3.map_partitions(
    find_stats_needs_meta,
    include_pixel=True,
    # Keyword arguments after 'include_pixel=' are passed to your function.
    target_column="phot_g_mean_mag",
    # Here we give Dask the hint it needs to avoid giving us an empty frame
    meta=output_meta,
)

In [15]:
%%time
npartitions = gaia3.npartitions
from dask.distributed import Client

with Client(n_workers=min(8, npartitions)) as client:
    result = unrealized.compute()
result

CPU times: user 772 ms, sys: 674 ms, total: 1.45 s
Wall time: 22.8 s


,pixel,phot_g_mean_mag_min,phot_g_mean_mag_max,phot_g_mean_mag_mean
0,"Order: 4, Pixel: 2944",8.154537,22.033451,19.083840
0,"Order: 4, Pixel: 2945",5.508409,22.345750,19.094207
0,"Order: 4, Pixel: 2946",6.264917,22.076380,19.102860
0,"Order: 4, Pixel: 2947",6.154421,21.986425,19.119143


The objects in the 'pixel' column are the same type as from `get_healpix_pixels()`.

In [16]:
type(result["pixel"].iloc[0])

hats.pixel_math.healpix_pixel.HealpixPixel

Because the result is one row per partition, we need additional reduction to get our single answer.

In [17]:
result["phot_g_mean_mag_min"].min(), result["phot_g_mean_mag_max"].max(), result[
    "phot_g_mean_mag_mean"
].mean()

(np.float64(5.508409), np.float64(22.34575), np.float64(19.100012538364453))

What about searching not only the four partitions from our cone search, but the whole catalog?  All that changes is the number of partitions.

**NOTE** that since we're using the `find_stats` that doesn't need `meta=`, we don't need to provide it.

In [18]:
gaia3_all = lsdb.open_catalog(
    f"{gaia_root}/gaia_dr3/gaia",
    margin_cache=f"{gaia_root}/gaia_dr3/gaia_10arcs",
    columns=[
        "source_id",
        "ra",
        "dec",
        "phot_g_mean_mag",
    ],
)
unrealized = gaia3_all.map_partitions(
    find_stats,
    include_pixel=True,
    # Keyword arguments after 'include_pixel=' are passed to your
    # function
    target_column="phot_g_mean_mag",
)
npartitions = unrealized.npartitions

In [19]:
npartitions

3933

That's a lot of partitions!  If we didn't bound this value, we could easily overwhelm our cluster.

This operation is light on compute, so let's give each worker several threads.

In [ ]:
%%time
with Client(n_workers=min(8, npartitions), memory_limit="4GB", threads_per_worker=4) as client:
    result = unrealized.compute()
result

2025-05-27 17:25:02,780 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.62 GiB -- Worker memory limit: 3.73 GiB


In [ ]:
# We need to do a final reduction step to get the true min and max
result["phot_g_mean_mag_min"].min(), result["phot_g_mean_mag_max"].max()

Since we just searched the whole catalog, we can check our answer
against the statistics that were compiled at import time for the
catalog.  As you can see, they match what we got when using
the `.map_partitions` method.

In [ ]:
gaia3_all.hc_structure.aggregate_column_statistics(include_columns="phot_g_mean_mag")

## 4. Histograms

Here's another example that requires a reduction and a change
in data shape.

Suppose we want to bin observations by magnitude in the G band.
We'll need to load different columns than before,
since we didn't include observations earlier.

This is more involved than the straightforward `pd.DataFrame.hist`
because each row contains some number of observations, and we want
to sum these observations within each magnitude bin.

In [ ]:
import lsdb

gaia3 = lsdb.open_catalog(
    f"{gaia_root}/gaia_dr3/gaia",
    margin_cache=f"{gaia_root}/gaia_dr3/gaia_10arcs",
    search_filter=lsdb.ConeSearch(ra=280, dec=-60, radius_arcsec=2 * 3600),
    columns=[
        "source_id",
        "ra",
        "dec",
        "phot_g_mean_mag",
        "phot_g_n_obs",
    ],
)
gaia3

### 4.1. All Within Memory

First, what does it look like to do it all within memory?

In [ ]:
%%time
df = gaia3.compute()
df

Note that the four partitions become over 450k rows once they are computed.  The lazily-loaded
catalog can't show you how many total rows there are before computing, only the number of
partitions.

In [ ]:
from matplotlib import pyplot as plt

mag_bins = list(range(5, 25, 1))
plt.hist(df["phot_g_mean_mag"], bins=mag_bins, weights=df["phot_g_n_obs"])
plt.yscale("log")

This is a small enough amount of data that it's easy to handle it
in-memory.  But we wouldn't be able to do this on the whole Gaia3
catalog.  For that, we'll need to use `.map_partitions`, and we'll
test this function on the same small piece of data before launching
it across the entire catalog.

The first step is to understand that we'll be creating partial histograms,
that is, a histogram for each partition, and we'll need to reduce those
down to a single histogram at the end.  One consequence of this is
the need to have the same number of bins in each partial histogram,
`mag_bins` in this case.  Often when producing a histogram, it's common
to let Pandas or `pd.cut` pick the bin boundaries given a number of
bins, but that will work against us in this case, since each partition
will have a different min and max, and the partial histograms won't
reduce.

In [ ]:
# This function only requires and returns one DataFrame: no pixel
# argument, no additional arguments.
def observation_histogram(df):
    df["binned"] = pd.cut(df["phot_g_mean_mag"], mag_bins)
    binned_data = df.groupby("binned", observed=True)["phot_g_n_obs"].sum()
    return pd.DataFrame(binned_data)

First, we test our new function on our single in-memory `DataFrame`, to
see that we get the results we expect.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()

bd = observation_histogram(df)

bd.plot(kind="bar", ax=ax, label="Observations")

ax.set_title("Histogram of Detections by Magnitude Bins")
ax.set_xlabel("G Band Magnitudes")
ax.set_ylabel("Total Sources")

ax.legend()
ax.set_yscale("log")

plt.show()

### 4.2. Using map_partitions

The above went pretty quickly, because our cone search is keeping our data small.
But if it were a much larger slice of the sky, this next technique would matter
more.

As before, `.map_partitions` will give us one result per partition, so we need
to be prepared to do a final combination step.

In [ ]:
unrealized = gaia3.map_partitions(observation_histogram)
unrealized

In [ ]:
%%time
npartitions = unrealized.npartitions
from dask.distributed import Client

with Client(n_workers=min(8, npartitions)) as client:
    result = unrealized.compute()
# This is actually the histogram repeated for each partition.
result

In [ ]:
# Final reduction step
total_histogram = result.groupby("binned", observed=True).sum()

import matplotlib.pyplot as plt

fig, ax = plt.subplots()
total_histogram.plot(kind="bar", ax=ax)
ax.set_title("Histogram of Detections by Magnitude")
ax.set_xlabel("G Band Magnitude")
ax.set_ylabel("Sum of Detections")
ax.set_yscale("log")
plt.show()

And now compare the result and timing to running this histogram on
the entire catalog.

In [ ]:
gaia3 = lsdb.open_catalog(
    f"{gaia_root}/gaia_dr3/gaia",
    margin_cache=f"{gaia_root}/gaia_dr3/gaia_10arcs",
    columns=[
        "source_id",
        "ra",
        "dec",
        "phot_g_mean_mag",
        "phot_g_n_obs",
    ],
)
unrealized = gaia3.map_partitions(observation_histogram)
unrealized

In [ ]:
%%time
npartitions = unrealized.npartitions
from dask.distributed import Client

# Takes 8-10 minutes
with Client(n_workers=min(8, npartitions), threads_per_worker=4) as client:
    result = unrealized.compute()
# This is actually the histogram repeated for each partition.
result

In [ ]:
# Final reduction
total_histogram = result.groupby("binned", observed=True).sum()

import matplotlib.pyplot as plt

fig, ax = plt.subplots()
total_histogram.plot(kind="bar", ax=ax)
ax.set_title("Histogram of Detections by Magnitude")
ax.set_xlabel("G Band Magnitude")
ax.set_ylabel("Sum of Detections")
ax.set_yscale("log")
plt.show()

The histogram has the same shape but a different Y-axis, as is reasonable:
our small cone search had picked a part of the sky which was representative
of the whole, but processing the entire catalog increased the number
of observations across all bins.

## About

**Authors**: Derek Jones

**Last updated on**: April 17, 2025

If you use `lsdb` for published research, please cite following [instructions](https://docs.lsdb.io/en/stable/citation.html).